In [14]:
# imports trlx from TIH
from TIH import trlx

In [20]:
# from trlx.data.default_configs import TRLConfig, default_ilql_confi
TRLConfig = trlx.data.default_configs.TRLConfig
default_ilql_config = trlx.data.default_configs.default_ilql_config

In [10]:
# standard imports
import numpy as np

In [4]:
# imports utility function components
import generators
import discriminators
import fitness_functions
import utility_functions
import adversaries

In [5]:
# target prompt
prompt = "write me a college essay"

In [7]:
generator = generators.GPT2()
discriminator = discriminators.BERT()
fitness_function = fitness_functions.PaperPrGen()
utility_function = utility_functions.PDUF(generator, discriminator, fitness_function, prompt)
# adversary = adversaries.PD_RLHF(utility_function)

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
# config dict for training
default_config = default_ilql_config().to_dict()
default_config['train']['tracker'] = None
default_config['train']['batch_size'] = 16
default_config['train']['epochs'] = 10
config = TRLConfig.update(default_config, {})

# sets gpu!
config['cuda_device'] = 0

print(config)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 config = TRLConfig.update(default_config, {})                                               │
│    7                                                                                             │
│    8 # sets gpu!                                                                                 │
│ ❱  9 config['cuda_device'] = 0                                                                   │
│   10                                                                                             │
│   11 print(config)                                                                               │
│   12                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'TRLConfig' object does not support item assignment

In [25]:
reward_fn = lambda samples, **kwargs: np.average(np.array([utility_function.PA_u(sample) for sample in samples]))
config = {
    'cuda_device': 0 # Use GPU with ID 0
}
trainer = trlx.train('gpt2',
                     reward_fn
                     config,
                     )

/home/ec2-user/cheatGPT/trlx/trlx/trlx.py:57: UserWarning: Passing the `config` argument implicitly is depreciated, use oradapt some from `trlx/data/default_configs.py` instead
  warnings.warn(
[RANK 0] Initializing model: gpt2


[RANK 0] Collecting rollouts
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 reward_fn=lambda samples, **kwargs: np.average(np.array([utility_function.PA_u(sample) f     │
│ ❱ 2 trainer = trlx.train('gpt2', reward_fn)                                                      │
│   3                                                                                              │
│                                                                                                  │
│ /home/ec2-user/cheatGPT/trlx/trlx/trlx.py:101 in train                                           │
│                                                                                                  │
│    98 │   │   if eval_prompts is None:                                                           │
│    99 │   │   │   eval_prompts = prompts[:batch_size]                                            │
│   100 │   │                                                                                      │
│ ❱ 101 │   │   trainer.make_experience(config.method.num_rollouts)                                │
│   102 │                                                                                          │
│   103 │   # Offline training from the collected samples (e.g. SFT, ILQL)                         │
│   104 │   elif samples:                                                                          │
│                                                                                                  │
│ /home/ec2-user/cheatGPT/trlx/trlx/trainer/accelerate_ppo_trainer.py:316 in make_experience       │
│                                                                                                  │
│   313 │   │   │   │                                                                              │
│   314 │   │   │   │   exp_score_time = time()                                                    │
│   315 │   │   │   │   all_scores = torch.tensor(                                                 │
│ ❱ 316 │   │   │   │   │   self.reward_fn(                                                        │
│   317 │   │   │   │   │   │   samples=all_str_samples,                                           │
│   318 │   │   │   │   │   │   prompts=all_str_prompts,                                           │
│   319 │   │   │   │   │   │   outputs=all_str_outputs,                                           │
│                                                                                                  │
│ in <lambda>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 reward_fn=lambda samples, **kwargs: np.average(np.array([utility_function.PA_u(sample) f     │
│   2 trainer = trlx.train('gpt2', reward_fn)                                                      │
│   3                                                                                              │
│                                                                                                  │
│ in <listcomp>:1                                                                                  │
│                                                                                                  │
│ ❱ 1 reward_fn=lambda samples, **kwargs: np.average(np.array([utility_function.PA_u(sample) f     │
│   2 trainer = trlx.train('gpt2', reward_fn)                                                      │
│   3                                                                                              │
│                                                                                                  │
│ /home/ec2-user/cheatGPT/utility_functions/PDUF.py:22 in PA_